In [ ]:
# Parameters
CONTEXTLENGTH = 512  # will be overridden by papermill

this notebook loads timesfm2.5 and makes predictions using it. Input context data is loaded from data/interim/data.csv. results are saved to data/processed/prediction_matrices/50/.

In [1]:
import pandas as pd
import torch
import numpy as np
import logging
from pyprojroot import here

import sys
sys.path.append(str(here() / 'scripts'))
sys.path.append(str(here() / 'timesfm'))

import timesfm
from utils import (get_data,get_y_context, save)

In [ ]:
MAXMODELCONTEXT= 16384 # max context length for this model
MAXCONTEXT = 12840 # max context length for this dataset (2017-01-01 - 2018-06-19)
if CONTEXTLENGTH > MAXMODELCONTEXT:
    CONTEXTLENGTH = MAXMODELCONTEXT

if CONTEXTLENGTH > MAXCONTEXT:
    CONTEXTLENGTH = MAXCONTEXT

MODEL = 'timesfm'
TARGET = 'occ'
FEATURESET = 'u'
TESTNAME = f'{TARGET}-{MODEL}-{FEATURESET}-{CONTEXTLENGTH}'
TESTSTART = pd.Timestamp('20180620')
TESTSIZE = 365*24
PREDICTIONLENGTH = 24

In [ ]:
logging.basicConfig(
        format='%(asctime)s - %(message)s', 
        level=logging.INFO,
        filename=here() / f"logs/logger/{TESTNAME}.log",      
        filemode='w'
        )
logging.info('Starting test.')
logging.info(f'MODEL: {MODEL} TARGET: {TARGET} FEATURESET:'
     f'{FEATURESET} CONTEXTLENGTH: {CONTEXTLENGTH}')

In [3]:
data = get_data(TARGET, MODEL)
cd = {
        'occ' : 'Target:Occupancy',
        'arr' : 'Target:Arrivals'
    }
target_name = cd[TARGET]
y = data[target_name]
y_context = get_y_context(data, TARGET, CONTEXTLENGTH, TESTSTART)

y_context = y_context.to_numpy()

In [4]:
model = timesfm.TimesFM_2p5_200M_torch()
model.load_checkpoint()

/home/valtti/code/ed-ml-multivar/env-timesfm/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Fetching 4 files: 100%|██████████| 4/4 [00:00<00:00, 67378.38it/s]


In [5]:
model.compile(
    timesfm.ForecastConfig(
        max_context=CONTEXTLENGTH,
        max_horizon=PREDICTIONLENGTH,
        normalize_inputs=True,
        use_continuous_quantile_head=True,
        force_flip_invariance=True,
        infer_is_positive=True,
        fix_quantile_crossing=True,
    )
)
# list[np.ndarrays] -> tuple[np.ndarray, np.ndarray]:
point_forecast, quantile_forecast = model.forecast(
    horizon=24,
    inputs=y_context
)
print(point_forecast.shape)  # (batch_size, 24)
print(quantile_forecast.shape)  # (batch_size, 24, 10): mean, then 10th to 90th quantiles.

(365, 24)
(365, 24, 10)


In [6]:
save(model_name=MODEL, featureset_name=FEATURESET, target_name=TARGET, context_length=CONTEXTLENGTH,
    test_start=TESTSTART, y_pred=point_forecast)